In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [57]:
#importing all csv files
domarar = pd.read_csv('csv/blak-domarar.csv', sep=';', header=0)
einstaklingar = pd.read_csv('csv/blak-einstaklingar.csv', sep=';', header=0)
forsvarsmenn = pd.read_csv('csv/blak-forsvarsmenn.csv', sep=';', header=0)
lid = pd.read_csv('csv/blak-lid.csv', sep=';', header=0)
lidimoti = pd.read_csv('csv/blak-lidimoti.csv', sep=';', header=0)
lidsmenn = pd.read_csv('csv/blak-lidsmenn.csv', sep=';', header=0)
lidsstjorar = pd.read_csv('csv/blak-lidsstjorar.csv', sep=';', header=0)
thjalfarar = pd.read_csv('csv/blak-thjalfarar.csv', sep=';', header=0)
mot = pd.read_csv('csv/blak-mot.csv', sep=';', header=0)

In [ ]:
#print(domarar)
#print(einstaklingar)
#print(forsvarsmenn)
#print(lid)
#print(lidimoti)
#print(lidsmenn)
#print(lidsstjorar)
#print(mot)
#print(thjalfarar)

In [72]:
#print(domarar.describe())
print(einstaklingar.describe())
#print(forsvarsmenn.describe())
#print(lid.describe())
#print(lidimoti.describe())
#print(lidsmenn.describe())
#print(lidsstjorar.describe())
#print(mot.describe())
#print(thjalfarar.describe())

           EinstID  Heimilisfang1  Heimilisfang2  Heimilisfang3         Haed
count  4497.000000            0.0            0.0            0.0  2325.000000
mean   2310.163442            NaN            NaN            NaN    38.581075
std    1327.469491            NaN            NaN            NaN    75.465668
min       1.000000            NaN            NaN            NaN     0.000000
25%    1169.000000            NaN            NaN            NaN     0.000000
50%    2314.000000            NaN            NaN            NaN     0.000000
75%    3461.000000            NaN            NaN            NaN     0.000000
max    4592.000000            NaN            NaN            NaN   906.000000


In [70]:
#print(help(einstaklingar))

In [69]:
#domarar.dtypes
#einstaklingar.dtypes
#forsvarsmenn.dtypes
#lid.dtypes
#lidimoti.dtypes
#lidsmenn.dtypes
#lidsstjorar.dtypes
#mot.dtypes
#thjalfarar.dtypes

EinstID            int64
Nafn              object
Fdagur            object
Kyn               object
FelagISI          object
Netfang           object
Heimilisfang1    float64
Heimilisfang2    float64
Heimilisfang3    float64
Simi1             object
Simi2             object
Simi3             object
Timastimpill      object
Haed             float64
dtype: object

In [59]:
#FINAL STEP (run after everything is done):
#save as new csv inside csv/new
pd.DataFrame(domarar).to_csv("csv/new/blak-domarar.csv")
pd.DataFrame(einstaklingar).to_csv("csv/new/blak-einstaklingar.csv")
pd.DataFrame(forsvarsmenn).to_csv("csv/new/blak-forsvarsmenn.csv.csv")
pd.DataFrame(lid).to_csv("csv/new/blak-lid.csv")
pd.DataFrame(lidimoti).to_csv("csv/new/blak-lidimoti.csv")
pd.DataFrame(lidsmenn).to_csv("csv/new/blak-lidsmenn.csv")
pd.DataFrame(lidsstjorar).to_csv("csv/new/blak-lidsstjorar.csv")
pd.DataFrame(mot).to_csv("csv/new/blak-mot.csv")
pd.DataFrame(thjalfarar).to_csv("csv/new/blak-thjalfarar.csv")